In [60]:
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import json
import os
import re
import pprint
import pandas as pd
import numpy as np

basepath = Path(os.getcwd())
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
    basepath =  Path(os.getcwd())
    print(os.getcwd())
load_dotenv(find_dotenv())

from src.preprocessing.split.train_test_split import train_test_split

In [7]:
logfile_path = basepath / "logs" / "experiment-log.json"
logfile_path

PosixPath('/srv/idp-radio-1/logs/experiment-log.json')

In [8]:
with open(logfile_path, 'r') as f:
            main_data = json.load(f)

experiments = main_data['experiments']

In [48]:
experiments_df = pd.DataFrame.from_dict(experiments)
records_struct = json.loads(experiments_df.to_json(orient="records"))    
experiments_flat = pd.json_normalize (records_struct) #use pd.io.json
experiments_flat.columns

Index(['id', 'name', 'filename', 'version', 'description',
       'classification_report', 'history.loss', 'history.auc',
       'history.precision', 'history.recall', 'history.f2_score',
       'history.binary_accuracy', 'history.val_loss', 'history.val_auc',
       'history.val_precision', 'history.val_recall', 'history.val_f2_score',
       'history.val_binary_accuracy', 'history.lr', 'test.loss', 'test.AUC',
       'test.Precision', 'test.Recall', 'test.F2Score', 'test.BinaryAccuracy',
       'benchmark.benchmark_name', 'benchmark.dataset_name',
       'benchmark.dataset_folder', 'benchmark.models_dir', 'benchmark.epochs',
       'benchmark.optimizer', 'benchmark.loss', 'benchmark.metrics',
       'benchmark.label_columns', 'benchmark.path_column',
       'benchmark.path_column_prefix', 'benchmark.shuffle',
       'benchmark.batch_size', 'benchmark.dim', 'benchmark.n_channels',
       'benchmark.nan_replacement', 'benchmark.unc_value', 'benchmark.u_enc',
       'benchmark.drop_last

In [88]:
experiments_flat["history.val_auc_max"] = experiments_flat["history.val_auc"].apply(lambda x:  max(x) if isinstance(x, list) else -np.inf)
experiments_flat = experiments_flat.sort_values("history.val_auc_max", ascending=False)

In [85]:
experiments_flat.iloc[0]["name"]

'InceptionV3_Chexpert_BCE_E10_B128'

In [92]:
def print_report(x):
    print(x["name"])
    print("history.val_auc_max: ", x["history.val_auc_max"])
    print(x.classification_report) 
    print("\n\n")
    

In [93]:
experiments_flat.apply(print_report, axis=1)

DenseNet121_Chexpert_BCE_E20_B32
history.val_auc_max:  0.786714673
                            precision    recall  f1-score   support

                No Finding       0.09      0.08      0.08      4408
Enlarged Cardiomediastinum       0.04      0.00      0.00      2204
              Cardiomegaly       0.10      0.09      0.10      5347
              Lung Opacity       0.48      0.54      0.51     21081
               Lung Lesion       0.03      0.00      0.01      1772
                     Edema       0.24      0.14      0.18     10541
             Consolidation       0.07      0.01      0.01      2874
                 Pneumonia       0.19      0.00      0.01      1148
               Atelectasis       0.16      0.00      0.01      6700
              Pneumothorax       0.09      0.04      0.06      4021
          Pleural Effusion       0.39      0.40      0.39     17264
             Pleural Other       0.00      0.00      0.00       726
                  Fracture       0.06      0.01 

5     None
22    None
16    None
4     None
7     None
13    None
8     None
6     None
3     None
12    None
20    None
14    None
23    None
11    None
15    None
21    None
9     None
0     None
1     None
19    None
18    None
10    None
17    None
2     None
dtype: object